# Pydantic Crash Course - Data Validation in Python

## Overview

**Pydantic** is a Python library for data validation and settings management. It's essential for production-grade code, especially when building APIs, working with configuration files, or creating ML pipelines.

**Why Learn Pydantic?**

- Used extensively in FastAPI
- Required for production-level code
- Common in data science ML pipelines
- Industry-standard for data validation

## The Two Big Problems Pydantic Solves

### Problem 1: Type Validation

### The Challenge in Python

Python is a **dynamically typed language** with no built-in static typing:

In [1]:
# Python allows this (problematic in production)
x = 10        # x is integer
x = "hello"   # Now x is string - no error!


### Real-World Problem Example

**Scenario**: Senior programmer creates a function to insert patient data into database.

In [2]:
# Senior programmer's code
def insert_patient_data(name, age):
    # Imagine database insertion code here
    print(f"Inserting into database...")
    print(f"Name: {name}")
    print(f"Age: {age}")

# Junior programmer uses the function
insert_patient_data("Nitish", "30")  # Bug! Age is string, not int


Inserting into database...
Name: Nitish
Age: 30


Problem: Code runs successfully but inserts wrong data type into database! ❌

### Attempted Solution 1: Type Hinting

In [3]:
def insert_patient_data(name: str, age: int):
    print(f"Inserting into database...")
    print(f"Name: {name}")
    print(f"Age: {age}")

# This still works even with wrong type!
insert_patient_data("Nitish", "30")  # No error - type hints don't enforce


Inserting into database...
Name: Nitish
Age: 30


**Limitation**: Type hints are **informational only** - they don't produce errors or enforce types.

### Attempted Solution 2: Manual Type Checking

In [4]:
def insert_patient_data(name: str, age: int):
    # Manual type validation
    if type(name) == str and type(age) == int:
        print(f"Inserting into database...")
        print(f"Name: {name}")
        print(f"Age: {age}")
    else:
        raise TypeError("Incorrect data type")

# Now this raises an error ✓
insert_patient_data("Nitish", "30")  # TypeError!


TypeError: Incorrect data type

**Problem with Manual Validation**:

- ❌ Not scalable
- ❌ Boilerplate code repeated everywhere
- ❌ Hard to maintain as fields increase

**Example of Scaling Problem**:

In [5]:
# Function 1: Insert
def insert_patient_data(name: str, age: int):
    if type(name) == str and type(age) == int:
        # Insert logic
        pass
    else:
        raise TypeError("Incorrect data type")

# Function 2: Update (same validation code!)
def update_patient_data(name: str, age: int):
    if type(name) == str and type(age) == int:  # Duplicated!
        # Update logic
        pass
    else:
        raise TypeError("Incorrect data type")

# Adding a new field (weight) requires updating ALL functions!


### Problem 2: Data Validation

Beyond type checking, you need **business logic validation**:

In [6]:
def insert_patient_data(name: str, age: int):
    # Type validation
    if type(name) == str and type(age) == int:
        # Data validation - age can't be negative
        if age < 0:
            raise ValueError("Age cannot be negative")
        else:
            # Insert logic
            print(f"Inserting {name}, age {age}")
    else:
        raise TypeError("Incorrect data type")


**More Validation Requirements**:

- Email must be valid format
- Phone number must follow pattern
- Age must be in range (0-120)
- Name max length 50 characters
- And many more...

**Result**: Massive boilerplate code that needs to be repeated across all functions! ❌

## How Pydantic Solves These Problems

### The Three-Step Pydantic Workflow

```
Step 1: Build Model (Define Schema)
         ↓
Step 2: Instantiate with Raw Data (Automatic Validation)
         ↓
Step 3: Use Validated Object in Functions

```

### Step 1: Build Pydantic Model

**Define the ideal schema** using a Pydantic class:

In [7]:
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int


**What this does**:

- Defines required fields
- Specifies data types
- Sets validation rules (more on this later)

### Step 2: Instantiate Model with Raw Data

**Create object from raw data** - validation happens automatically:

In [8]:
# Raw input data (could come from API, form, file, etc.)
patient_data = {
    "name": "Nitish",
    "age": 30
}

# Create Pydantic object (validation happens here!)
patient1 = Patient(**patient_data)


**What happens during instantiation**:

- ✅ Checks if all required fields are present
- ✅ Validates data types
- ✅ Applies any validation rules
- ✅ Converts types automatically when possible
- ❌ Raises ValidationError if anything fails

### Step 3: Use Validated Object

**Pass validated object to your function**:

In [9]:
def insert_patient_data(patient: Patient):
    # Now we're guaranteed correct data!
    print(f"Inserting into database...")
    print(f"Name: {patient.name}")
    print(f"Age: {patient.age}")

# Use the validated object
insert_patient_data(patient1)


Inserting into database...
Name: Nitish
Age: 30


## Complete Example: Before vs After Pydantic

### Before Pydantic (Manual Validation)

In [11]:
def insert_patient_data(name: str, age: int):
    # Manual type validation
    if type(name) == str and type(age) == int:
        # Manual data validation
        if age < 0:
            raise ValueError("Age cannot be negative")
        else:
            print(f"Inserting {name}, age {age}")
    else:
        raise TypeError("Incorrect data type")

def update_patient_data(name: str, age: int):
    # Same validation code repeated!
    if type(name) == str and type(age) == int:
        if age < 0:
            raise ValueError("Age cannot be negative")
        else:
            print(f"Updating {name}, age {age}")
    else:
        raise TypeError("Incorrect data type")


In [12]:
insert_patient_data("Nitish", 30)      # Works

Inserting Nitish, age 30


In [13]:
insert_patient_data("Nitish", "30")    # TypeError

TypeError: Incorrect data type

In [14]:
insert_patient_data("Nitish", -5)      # ValueError

ValueError: Age cannot be negative

### After Pydantic (Clean & Scalable)

In [15]:
from pydantic import BaseModel, Field

# Step 1: Define schema ONCE
class Patient(BaseModel):
    name: str
    age: int = Field(gt=0)  # Greater than 0

# Step 2: Functions use the model
def insert_patient_data(patient: Patient):
    print(f"Inserting {patient.name}, age {patient.age}")

def update_patient_data(patient: Patient):
    print(f"Updating {patient.name}, age {patient.age}")

# Step 3: Create validated objects
patient_data = {"name": "Nitish", "age": 30}
patient1 = Patient(**patient_data)


In [16]:
# Use in functions
insert_patient_data(patient1)  # Works ✓
update_patient_data(patient1)  # Works ✓

Inserting Nitish, age 30
Updating Nitish, age 30


In [18]:
# This will fail validation automatically
bad_data = {"name": "Nitish", "age": "30"}
patient2 = Patient(**bad_data)  # Auto-converts "30" to 30!

insert_patient_data(patient2)  
update_patient_data(patient2)

Inserting Nitish, age 30
Updating Nitish, age 30


In [19]:
# This will raise ValidationError
invalid_data = {"name": "Nitish", "age": -5}
patient3 = Patient(**invalid_data)  # ValidationError: age must be > 0

insert_patient_data(patient3)
update_patient_data(patient3)

ValidationError: 1 validation error for Patient
age
  Input should be greater than 0 [type=greater_than, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

**Benefits**:

- ✅ Validation logic defined once in model
- ✅ No repeated boilerplate code
- ✅ Easy to add new fields (update model only)
- ✅ Automatic type conversion when safe
- ✅ Clear error messages

## Building Complex Pydantic Models

### Basic Data Types

In [20]:
from pydantic import BaseModel

class Patient(BaseModel):
    name: str           # String
    age: int            # Integer
    weight: float       # Float
    height: float       # Float
    bmi: float         # Float
    is_married: bool   # Boolean


### Complex Data Types: Lists and Dictionaries

**Problem**: Can't use built-in `list` and `dict` directly for type validation.

In [21]:
# ❌ Wrong - only validates it's a list, not contents
class Patient(BaseModel):
    allergies: list  # Only checks if it's a list


Solution: Import from typing module for nested type validation:

In [22]:
from typing import List, Dict
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int
    weight: float
    height: float
    bmi: float
    is_married: bool
    allergies: List[str]              # List of strings
    contact_details: Dict[str, str]   # Dictionary with string keys & values


**Why use `List[str]` instead of `list`?**

- `list` only validates that it's a list
- `List[str]` validates:
    - ✅ It's a list
    - ✅ Every item inside is a string

In [23]:
from typing import List, Dict
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int
    weight: float
    is_married: bool
    allergies: List[str]
    contact_details: Dict[str, str]

# Create patient object
patient_data = {
    "name": "Nitish",
    "age": 30,
    "weight": 75.2,
    "is_married": True,  # or 1 (auto-converts)
    "allergies": ["Pollen", "Dust"],
    "contact_details": {
        "email": "abc@gmail.com",
        "phone": "1234567890"
    }
}

patient = Patient(**patient_data)
print(patient.name)      # Nitish
print(patient.allergies) # ['Pollen', 'Dust']


Nitish
['Pollen', 'Dust']


#### Validation in Action:

In [24]:
# ❌ This will fail - integer in allergies list
bad_data = {
    "name": "John",
    "age": 25,
    "weight": 70.0,
    "is_married": False,
    "allergies": ["Pollen", 123],  # Integer not allowed!
    "contact_details": {"email": "test@test.com", "phone": "999"}
}

patient = Patient(**bad_data)  # ValidationError!



ValidationError: 1 validation error for Patient
allergies.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

In [25]:
# ❌ This will fail - integer value in dictionary
bad_data2 = {
    "name": "John",
    "age": 25,
    "weight": 70.0,
    "is_married": False,
    "allergies": ["Pollen"],
    "contact_details": {
        "email": "test@test.com",
        "phone": 1234567890  # Integer not allowed!
    }
}

patient2 = Patient(**bad_data2)  # ValidationError!

ValidationError: 1 validation error for Patient
contact_details.phone
  Input should be a valid string [type=string_type, input_value=1234567890, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

## Required vs Optional Fields

### Required Fields (Default Behavior)

**All fields are required by default** in Pydantic models:

In [26]:
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int
    weight: float

# ❌ This fails - missing 'weight'
patient_data = {"name": "John", "age": 30}
patient = Patient(**patient_data)  # ValidationError: field required


ValidationError: 1 validation error for Patient
weight
  Field required [type=missing, input_value={'name': 'John', 'age': 30}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

### Making Fields Optional

**Use `Optional` from typing module**:

In [27]:
from typing import Optional, List
from pydantic import BaseModel

class Patient(BaseModel):
    name: str              # Required
    age: int               # Required
    weight: float          # Required
    is_married: bool       # Required
    allergies: Optional[List[str]] = None  # Optional with default None


**Syntax Breakdown**:

- `Optional[Type]` - Marks field as optional
- `= None` - Sets default value (required for optional fields)

**Example**:

In [29]:
from typing import Optional, List
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int
    allergies: Optional[List[str]] = None
    is_married: Optional[bool] = False  # Default is False


In [30]:
# Without allergies (uses default None)
patient1 = Patient(name="John", age=30)
print(patient1.allergies)    # None
print(patient1.is_married)   # False

None
False


In [31]:
# With allergies
patient2 = Patient(name="Jane", age=25, allergies=["Dust"])
print(patient2.allergies)    # ['Dust']

['Dust']


### Setting Default Values

You can set defaults for any field (not just optional ones):

In [32]:
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int
    is_married: bool = False  # Default value
    country: str = "USA"      # Default value

# Without specifying defaults
patient = Patient(name="John", age=30)
print(patient.is_married)  # False
print(patient.country)     # USA


False
USA


In [33]:
# Overriding defaults
patient2 = Patient(name="Jane", age=25, is_married=True, country="UK")
print(patient2.is_married)  # True
print(patient2.country)     # UK

True
UK


## Data Validation Methods

### Method 1: Custom Data Types from Pydantic

Pydantic provides **built-in data types** for common validation scenarios:

### EmailStr - Email Validation

In [39]:
#!pip install email-validator    # Installs the required dependency for EmailStr validation
!pip install "pydantic[email]"   # Installs Pydantic along with email validation support

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 13.1 MB/s eta 0:00:00


In [40]:
from pydantic import BaseModel, EmailStr

class Patient(BaseModel):
    name: str
    email: EmailStr  # Validates email format

# Valid email
patient1 = Patient(name="John", email="john@gmail.com")  # ✓


In [41]:
# Invalid email
patient2 = Patient(name="Jane", email="notanemail")  # ValidationError
patient3 = Patient(name="Bob", email="bob@invalid")  # ValidationError

ValidationError: 1 validation error for Patient
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='notanemail', input_type=str]

**What EmailStr validates**:

- Contains `@` symbol
- Valid email format
- Proper domain structure


### AnyUrl - URL Validation


In [42]:
from pydantic import BaseModel, AnyUrl

class Patient(BaseModel):
    name: str
    linkedin_url: AnyUrl  # Validates URL format

# Valid URLs
patient1 = Patient(
    name="John",
    linkedin_url="https://linkedin.com/in/john"
)  # ✓

patient2 = Patient(
    name="Jane",
    linkedin_url="http://example.com"
)  # ✓



In [43]:
# Invalid URLs
patient3 = Patient(
    name="Bob",
    linkedin_url="linkedin.com"  # Missing http://
)  # ValidationError

patient4 = Patient(
    name="Alice",
    linkedin_url="not a url"
)  # ValidationError

ValidationError: 1 validation error for Patient
linkedin_url
  Input should be a valid URL, relative URL without a base [type=url_parsing, input_value='linkedin.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/url_parsing

**What AnyUrl validates**:

- Proper URL protocol (`http://`, `https://`, `file://`, etc.)
- Valid URL structure
- Can be web URL or file URL

### Other Built-in Types

In [45]:
from pydantic import (
    BaseModel,
    EmailStr,
    AnyUrl,
    HttpUrl,      # Only HTTP/HTTPS URLs
    FilePath,     # Valid file path
    DirectoryPath, # Valid directory path
    IPvAnyAddress, # IP address validation
    PositiveInt,  # Integer > 0
    NegativeInt,  # Integer < 0
    constr,       # Constrained string
    conint        # Constrained integer
)

class AdvancedPatient(BaseModel):
    email: EmailStr
    website: HttpUrl
    age: PositiveInt

patient = AdvancedPatient(
    email="patient@example.com",
    website="https://example.com",
    age=30
)

print(patient)


email='patient@example.com' website=HttpUrl('https://example.com/') age=30


### Method 2: Field Function for Custom Validation

The `Field()` function allows **custom validation rules** based on your business logic

In [46]:
from pydantic import BaseModel, Field

class Patient(BaseModel):
    name: str
    age: int
    weight: float


### Numeric Constraints

**Greater Than (gt)**:

In [48]:
from pydantic import BaseModel, Field

class Patient(BaseModel):
    weight: float = Field(gt=0)  # Weight must be > 0

patient1 = Patient(weight=70.5)   # ✓


In [50]:
patient2 = Patient(weight=-10)    # ValidationError: must be > 0

ValidationError: 1 validation error for Patient
weight
  Input should be greater than 0 [type=greater_than, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

In [51]:
patient3 = Patient(weight=0)      # ValidationError: must be > 0

ValidationError: 1 validation error for Patient
weight
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

**Greater Than or Equal (ge):**

In [52]:
class Patient(BaseModel):
    age: int = Field(ge=0)  # Age must be >= 0

patient = Patient(age=0)   # ✓


In [53]:
atient = Patient(age=-1)  # ValidationError

ValidationError: 1 validation error for Patient
age
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal

 **Same in less than (lt), less than equal to (lte)**

#### **Range Validation (Combining gt/lt or ge/le):**

In [54]:
class Patient(BaseModel):
    age: int = Field(gt=0, lt=120)  # 0 < age < 120
    bmi: float = Field(ge=10.0, le=50.0)  # 10 <= BMI <= 50

patient1 = Patient(age=30, bmi=22.5)  # ✓


In [55]:
patient2 = Patient(age=0, bmi=22.5)   # ValidationError: age must be > 0
patient3 = Patient(age=150, bmi=22.5) # ValidationError: age must be < 120
patient4 = Patient(age=30, bmi=5.0)   # ValidationError: BMI must be >= 10

ValidationError: 1 validation error for Patient
age
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

### String Constraints

**Max Length**:

In [57]:
class Patient(BaseModel):
    name: str = Field(max_length=50)  # Max 50 characters

patient1 = Patient(name="John Doe")  # ✓



In [58]:
patient2 = Patient(name="A" * 100)   # ValidationError: max 50 chars

ValidationError: 1 validation error for Patient
name
  String should have at most 50 characters [type=string_too_long, input_value='AAAAAAAAAAAAAAAAAAAAAAAA...AAAAAAAAAAAAAAAAAAAAAAA', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_long

#### **same in Min Length (min_length) and Exact Length (exact_length)**

### List Constraints

**Max Items**:

In [59]:
from typing import List
from pydantic import BaseModel, Field

class Patient(BaseModel):
    allergies: List[str] = Field(max_length=5)  # Max 5 allergies

patient1 = Patient(allergies=["Pollen", "Dust"])  # ✓

In [60]:
patient2 = Patient(allergies=["A", "B", "C", "D", "E", "F"])  # ValidationError

ValidationError: 1 validation error for Patient
allergies
  List should have at most 5 items after validation, not 6 [type=too_long, input_value=['A', 'B', 'C', 'D', 'E', 'F'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/too_long

**Min Items**:

In [62]:
class Patient(BaseModel):
    medications: List[str] = Field(min_length=1)  # At least 1 medication

patient1 = Patient(medications=["Aspirin"])  # ✓


In [63]:
patient2 = Patient(medications=[])  # ValidationError: need at least 1

ValidationError: 1 validation error for Patient
medications
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/too_short

### **Complete Example with Multiple Constraints**

In [ ]:
from typing import List, Optional, Dict
from pydantic import BaseModel, Field, EmailStr

class Patient(BaseModel):
    # String with max length
    name: str = Field(max_length=50)

    # Integer with range
    age: int = Field(gt=0, lt=120)

    # Float with range
    weight: float = Field(gt=0, description="Weight in kg")
    height: float = Field(gt=0, le=300, description="Height in cm")
    bmi: float = Field(ge=10.0, le=50.0)

    # Email validation
    email: EmailStr

    # List with max items
    allergies: Optional[List[str]] = Field(default=None, max_length=10)

    # Optional with default
    is_married: bool = False

# Valid patient
patient_data = {
    "name": "John Doe",
    "age": 30,
    "weight": 75.5,
    "height": 175.0,
    "bmi": 24.7,
    "email": "john@example.com",
    "allergies": ["Pollen", "Dust"]
}

patient = Patient(**patient_data)  # ✓

In [ ]:
# Invalid examples
patient_data["age"] = -5          # ValidationError: age must be > 0
patient_data["weight"] = 0        # ValidationError: weight must be > 0
patient_data["bmi"] = 60          # ValidationError: BMI must be <= 50
patient_data["email"] = "invalid" # ValidationError: invalid email
patient_data["allergies"] = ["A"] * 15  # ValidationError: max 10 items

## Field() Function: Adding Metadata

Beyond validation, `Field()` is used to **add descriptions and documentation**:

In [70]:
from pydantic import BaseModel, Field

class Patient(BaseModel):
    name: str = Field(
        max_length=50,
        description="Full name of the patient"
    )

    age: int = Field(
        gt=0,
        lt=120,
        description="Patient's age in years",
        example=30
    )

    email: EmailStr = Field(
        description="Patient's email address",
        example="patient@example.com"
    )


/tmp/ipython-input-758112977.py:9: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  age: int = Field(
/tmp/ipython-input-758112977.py:16: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  email: EmailStr = Field(


**Where metadata is useful**:

- **API Documentation**: Appears in FastAPI's auto-generated docs
- **Code Readability**: Helps other developers understand fields
- **IDE Support**: Better autocomplete and hints

**Example in FastAPI docs**:
When you build an API with FastAPI, this metadata appears in the `/docs` page, helping clients understand what each field expects.

## Automatic Type Conversion

Pydantic automatically converts types **when safe to do so**:

In [71]:
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: int

# String "30" automatically converted to int 30
patient_data = {"name": "John", "age": "30"}
patient = Patient(**patient_data)

print(patient.age)        # 30 (int, not string!)
print(type(patient.age))  # <class 'int'>


30
<class 'int'>


##### **Safe Conversions:**

In [ ]:

# String to int (if valid number)
class Patient(BaseModel):
    age: int
Patient(age="30")      # ✓ → 30

# Float to int (truncates)
class Patient(BaseModel):
    age: int
Patient(age=30.9)  # ✓ → 30

# String to bool
class Patient(BaseModel):
    is_married: bool
Patient(is_married="true")   # ✓ → True
Patient(is_married="1")      # ✓ → True
Patient(is_married="false")  # ✓ → False
Patient(is_married="0")      # ✓ → False

# Int to bool
Patient(is_married=1)   # ✓ → True
Patient(is_married=0)   # ✓ → False




Patient(is_married=False)

#### **Unsafe Conversions (Will Fail):**

In [ ]:
# Float to int (truncates)
class Patient(BaseModel):
    age: int
Patient(age=30.9)  # ✓ → 30 (in previous versions but not in Pydantic v2 )

# allowed in previous versions but not in Pydantic v2   

ValidationError: 1 validation error for Patient
age
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=30.9, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/int_from_float

In [78]:
# Can't convert invalid string to int
class Patient(BaseModel):
    age: int
Patient(age="thirty")  # ✗ ValidationError



ValidationError: 1 validation error for Patient
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

In [80]:
# Can't convert random string to bool
class Patient(BaseModel):
    is_married: bool
Patient(is_married="yes he is marries")  # ✗ ValidationError

ValidationError: 1 validation error for Patient
is_married
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value='yes he is marries', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/bool_parsing

### **Complete Working Example**

In [81]:
from typing import List, Optional, Dict
from pydantic import BaseModel, Field, EmailStr, AnyUrl

class Patient(BaseModel):
    # Basic fields with validation
    name: str = Field(
        max_length=50,
        description="Patient's full name"
    )

    age: int = Field(
        gt=0,
        lt=120,
        description="Patient's age in years"
    )

    weight: float = Field(
        gt=0,
        description="Weight in kilograms"
    )

    height: float = Field(
        gt=0,
        le=300,
        description="Height in centimeters"
    )

    bmi: float = Field(
        ge=10.0,
        le=50.0,
        description="Body Mass Index"
    )

    # Built-in validators
    email: EmailStr = Field(description="Patient's email")
    linkedin_url: Optional[AnyUrl] = None

    # Optional fields
    is_married: Optional[bool] = False
    allergies: Optional[List[str]] = Field(
        default=None,
        max_length=5,
        description="List of allergies"
    )

    # Complex types
    contact_details: Dict[str, str] = Field(
        description="Contact information"
    )

# Function using the model
def insert_patient_data(patient: Patient):
    print(f"Inserting patient: {patient.name}")
    print(f"Age: {patient.age}")
    print(f"Email: {patient.email}")
    print(f"Allergies: {patient.allergies}")

# Valid data
patient_data = {
    "name": "John Doe",
    "age": 30,
    "weight": 75.5,
    "height": 175.0,
    "bmi": 24.7,
    "email": "john@example.com",
    "linkedin_url": "https://linkedin.com/in/johndoe",
    "is_married": True,
    "allergies": ["Pollen", "Dust"],
    "contact_details": {
        "phone": "1234567890",
        "address": "123 Main St"
    }
}

# Create validated patient object
patient = Patient(**patient_data)

# Use in function
insert_patient_data(patient)

# Access fields
print(patient.name)       # John Doe
print(patient.age)        # 30
print(patient.allergies)  # ['Pollen', 'Dust']


Inserting patient: John Doe
Age: 30
Email: john@example.com
Allergies: ['Pollen', 'Dust']
John Doe
30
['Pollen', 'Dust']


## Key Takeaways

### Why Use Pydantic?

1. **✅ Type Validation** - Enforces correct data types
2. **✅ Data Validation** - Enforces business rules (ranges, formats, etc.)
3. **✅ No Boilerplate** - Write validation rules once in model
4. **✅ Automatic Conversion** - Safe type conversions happen automatically
5. **✅ Clear Errors** - Helpful error messages when validation fails
6. **✅ Self-Documenting** - Models serve as documentation
7. **✅ IDE Support** - Better autocomplete and type hints

### When to Use Pydantic

- ✅ Building APIs (especially with FastAPI)
- ✅ Configuration file validation
- ✅ Data science ML pipelines
- ✅ Any production-grade Python code
- ✅ Parsing external data (JSON, forms, APIs)

### Best Practices

1. **Always use Pydantic V2** (not V1)
2. **Import from typing** for complex types (`List`, `Dict`, `Optional`)
3. **Use Field()** for validation constraints and metadata
4. **Use built-in types** (EmailStr, AnyUrl) when available
5. **Set sensible defaults** for optional fields
6. **Add descriptions** for better documentation

### Common Patterns

**Required fields**:

```python
name: str
age: int

```

**Optional fields**:

```python
allergies: Optional[List[str]] = None
is_married: Optional[bool] = False

```

**Fields with validation**:

```python
age: int = Field(gt=0, lt=120)
email: EmailStr

```

**Fields with metadata**:

```python
name: str = Field(max_length=50, description="Patient name")

```

## Summary Table

| **Feature** | **Without Pydantic** | **With Pydantic** |
| --- | --- | --- |
| Type checking | Manual `type()` checks | Automatic |
| Data validation | Manual if statements | Declarative with `Field()` |
| Code duplication | High (repeat in every function) | Low (define once) |
| Maintainability | Hard to update | Easy (change model only) |
| Type conversion | Manual | Automatic (when safe) |
| Error messages | Custom | Built-in and clear |
| Documentation | Separate docs needed | Self-documenting |